<a href="https://colab.research.google.com/github/leehanchung/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np

In [0]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"
column_names = ['Class Name', 
                'handicapped-infants', 
                'water-project-cost-sharing', 
                'adoption-of-the-budget-resolution',
                "physician-fee-freeze",
                "el-salvador-aid",
                'religious-groups-in-schools',
                'anti-satellite-test-ban',
                'aid-to-nicaraguan-contras',
                'mx-missile',
                'immigration',
                'synfuels-corporation-cutback',
                'education-spending',
                'superfund-right-to-sue',
                'crime',
                'duty-free-exports',
                'export-administration-act-south-africa']
pd.set_option('display.max_columns', 17)

In [0]:
df = pd.read_csv(url, header=None, names=column_names)

In [4]:
print(df.head())
print(df.describe())
df.isnull().any()

   Class Name handicapped-infants water-project-cost-sharing  \
0  republican                   n                          y   
1  republican                   n                          y   
2    democrat                   ?                          y   
3    democrat                   n                          y   
4    democrat                   y                          y   

  adoption-of-the-budget-resolution physician-fee-freeze el-salvador-aid  \
0                                 n                    y               y   
1                                 n                    y               y   
2                                 y                    ?               y   
3                                 y                    n               ?   
4                                 y                    n               y   

  religious-groups-in-schools anti-satellite-test-ban  \
0                           y                       n   
1                           y               

Class Name                                False
handicapped-infants                       False
water-project-cost-sharing                False
adoption-of-the-budget-resolution         False
physician-fee-freeze                      False
el-salvador-aid                           False
religious-groups-in-schools               False
anti-satellite-test-ban                   False
aid-to-nicaraguan-contras                 False
mx-missile                                False
immigration                               False
synfuels-corporation-cutback              False
education-spending                        False
superfund-right-to-sue                    False
crime                                     False
duty-free-exports                         False
export-administration-act-south-africa    False
dtype: bool

In [5]:
df['immigration'].value_counts()

y    216
n    212
?      7
Name: immigration, dtype: int64

In [0]:
df_clean = df.replace({'y':1, 'n':0, '?':np.nan})

In [7]:
print(df_clean.immigration.unique())
df_clean.immigration.value_counts()

[ 1.  0. nan]


1.0    216
0.0    212
Name: immigration, dtype: int64

In [8]:
df_clean.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [9]:
df_clean['Class Name'].unique()

array(['republican', 'democrat'], dtype=object)

In [0]:
gop = df_clean[df_clean['Class Name'] == 'republican']
dem = df_clean[df_clean['Class Name'] == 'democrat']

In [0]:
from scipy.stats import ttest_1samp, ttest_ind, levene

`scipy.stats.ttest_ind` does a two-sided test. p-value is computed to quantify the strength of the evidence, or confidence level.  Sample mean can be smaller or larger than the hypothesized mean. If it is smaller than the hypothesized value, then the t-statistic will be negative.  If it is larger, the t-statistic will be positive.


####NULL: The mean of support for the issue is greater in party a vs party b with p < 0.01.



###2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01


**NULL `H0`:** Republicans supports an issue less than or equal to  Democrats.

**ALT `H1`:** Republicans supports an issue more than Democrats.

Since `ttest_ind` is a two tail test, we would need to divide p value by 2 to turn it into a one tail test. Search through issues for p/2 < 0.01. If t > 0, then GOP support more than Democrats if we use `gop` as the first input to `ttest_ind`.  Note the independent t-test is:

$$ttest_{independent} = \frac{\mu_a - \mu_b}{\sqrt{{S^2/N_a + S^2/N_b}}}$$

So a positive t-test means first sample mean is higher than second sample mean.

References:

https://stackoverflow.com/questions/15984221/how-to-perform-two-sample-one-tailed-t-test-with-numpy-scipy

In [22]:
for issue in column_names[1:]:
    t, p = ttest_ind(gop[issue], dem[issue], nan_policy='omit')
    if t > 0 and p/2 < 0.01:
        print("GOP supports {} MORE than Democrats with t-stats of {:.2f} greater than 0 and confidence level of {:.2%}".format(issue, t, 1-p/2))

GOP supports physician-fee-freeze MORE than Democrats with t-stats of 49.37 greater than 0 and confidence level of 100.00%
GOP supports el-salvador-aid MORE than Democrats with t-stats of 21.14 greater than 0 and confidence level of 100.00%
GOP supports religious-groups-in-schools MORE than Democrats with t-stats of 9.74 greater than 0 and confidence level of 100.00%
GOP supports education-spending MORE than Democrats with t-stats of 20.50 greater than 0 and confidence level of 100.00%
GOP supports superfund-right-to-sue MORE than Democrats with t-stats of 13.51 greater than 0 and confidence level of 100.00%
GOP supports crime MORE than Democrats with t-stats of 16.34 greater than 0 and confidence level of 100.00%


###3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01


**NULL `H0`:** Democrats support an issue less than or equal to Republicans.

**ALT `H1`:** Democrats supports an issue greater than Republicans.


In [21]:
for issue in column_names[1:]:
    t, p = ttest_ind(dem[issue], gop[issue], nan_policy='omit')
    if t > 0 and p/2 < 0.01:
        print("Democrats supports {} more than Republicans with t-stats of {:.2f} greater than 0 and confidence level of {:.2%}".format(issue, t, 1-p/2))

Democrats supports handicapped-infants more than Republicans with t-stats of 9.21 greater than 0 and confidence level of 100.00%
Democrats supports adoption-of-the-budget-resolution more than Republicans with t-stats of 23.21 greater than 0 and confidence level of 100.00%
Democrats supports anti-satellite-test-ban more than Republicans with t-stats of 12.53 greater than 0 and confidence level of 100.00%
Democrats supports aid-to-nicaraguan-contras more than Republicans with t-stats of 18.05 greater than 0 and confidence level of 100.00%
Democrats supports mx-missile more than Republicans with t-stats of 16.44 greater than 0 and confidence level of 100.00%
Democrats supports synfuels-corporation-cutback more than Republicans with t-stats of 8.29 greater than 0 and confidence level of 100.00%
Democrats supports duty-free-exports more than Republicans with t-stats of 12.85 greater than 0 and confidence level of 100.00%
Democrats supports export-administration-act-south-africa more than Re

###4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

**NULL `H0`:** There's no diference between Republican and Democrat support on an issue.

**ALT `H1`:** There is a difference between Republican and Democrat support on an issue.

In [23]:
for issue in column_names[1:]:
    t, p = ttest_ind(dem[issue], gop[issue], nan_policy='omit')
    if p > 0.1:
        print("For {}, the p-value of {:.2} is greater than 0.1 between Republicans and Democrats.".format(issue, p))

For water-project-cost-sharing, the p-value of 0.93 is greater than 0.1 between Republicans and Democrats.
